In [1]:
import os
import shutil


from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
import numpy as np
from sklearn.utils import shuffle
np.random.seed(1)
from keras import backend as K

import h5py
import sys

from PIL import Image
sys.modules['Image'] = Image

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
# preprocessing, move files to current project folder use soft link to faster and reduce space usage

keras_download_train = '/home/ubuntu/.kaggle/competitions/dogs-vs-cats-redux-kernels-edition/train'
keras_download_test = '/home/ubuntu/.kaggle/competitions/dogs-vs-cats-redux-kernels-edition/test'
train_filenames_list = os.listdir(keras_download_train)

train_cat = list(filter(lambda x:x[:3] == 'cat', train_filenames_list))
train_dog = list(filter(lambda x:x[:3] == 'dog', train_filenames_list))


def check_make_dir(*dirs):
    print("example input ('folder1','folder2')")
    for count,dirname in enumerate(dirs):
        if os.path.exists(dirname) and os.path.isdir(dirname):
            shutil.rmtree(dirname)
        os.mkdir(dirname)
        
check_make_dir('train2','train2/cat','train2/dog','test2')

check_make_dir('train','train/cat','train/dog','test','validation','validation/cat','validation/dog')
        
        
os.symlink(keras_download_test, 'test2/test')

for filename in train_cat:
    os.symlink(keras_download_train+'/'+filename, 'train2/cat/'+filename)

for filename in train_dog:
    os.symlink(keras_download_train+'/'+filename, 'train2/dog/'+filename)
    

os.symlink(keras_download_test, 'test/test')

for filename in train_cat[:8500]:
    os.symlink(keras_download_train+'/'+filename, 'train/cat/'+filename)

for filename in train_dog[:8500]:
    os.symlink(keras_download_train+'/'+filename, 'train/dog/'+filename)
    
for filename in train_cat[8500:]:
    os.symlink(keras_download_train+'/'+filename, 'validation/cat/'+filename)

for filename in train_dog[8500:]:
    os.symlink(keras_download_train+'/'+filename, 'validation/dog/'+filename)
    

example input ('folder1','folder2')
example input ('folder1','folder2')


In [4]:
def get_input_shape(img_width, img_height):
    if K.image_data_format() == 'channels_first':
        input_shape = (3,img_width,img_height)
    else:
        input_shape = (img_width,img_height,3)
    return input_shape


In [2]:
# prepare image tensor

image_gen = ImageDataGenerator(featurewise_center=True,fill_mode='constant', 
                               cval=0, horizontal_flip=True, rotation_range=20, shear_range=0.2)

train_gen = image_gen.flow_from_directory("train2", (299, 299), shuffle=False, 
                                    batch_size=32, class_mode=None)

test_gen = image_gen.flow_from_directory("test2", (299, 299), shuffle=False, 
                                         batch_size=32, class_mode=None)

train_filenames = train_gen.filenames
train_nb_samples = len(train_filenames)

test_filenames = test_gen.filenames
test_nb_samples = len(test_filenames)


# define professor X
x = Input((299, 299, 3))
x = InceptionResNetV2(input_tensor=x, weights='imagenet', include_top=False, pooling = 'avg')


train = x.predict_generator(train_gen, steps=train_nb_samples, 
                                            verbose=1)

test = x.predict_generator(test_gen, steps=test_nb_samples, 
                                           verbose=1)
    

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras_preprocessing/image.py:959: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


12500/12500 [==============================] - 10371s 830ms/step


In [5]:
with h5py.File("gap_{0}.h5".format('InceptionResNetV2')) as h:
    h.create_dataset("train", data=train)
    h.create_dataset("test", data=test)
    h.create_dataset("label", data=train_gen.classes)
        

The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.


OSError: Can't prepare for writing data (file write failed: time = Thu Aug  2 10:03:41 2018
, filename = 'gap_InceptionResNetV2.h5', file descriptor = 63, errno = 28, error message = 'No space left on device', buf = 0x7ff4c6973fb0, total write size = 1868353632, bytes this sub-write = 1868353632, bytes actually written = 18446744073709551615, offset = 10408222720)

In [28]:
ls -ltrh

total 9.7G
-rw-rw-r-- 1 ubuntu ubuntu  19K Aug  1 23:58 Untitled.ipynb
-rw-rw-r-- 1 ubuntu ubuntu    6 Aug  1 23:58 README.md
-rw-rw-r-- 1 ubuntu ubuntu  91K Aug  1 23:58 model.h5_save
drwxrwxr-x 2 ubuntu ubuntu 4.0K Aug  1 23:58 uda/
-rw-rw-r-- 1 ubuntu ubuntu 112K Aug  1 23:58 sample_submission.csv
-rw-rw-r-- 1 ubuntu ubuntu 307K Aug  1 23:58 pred.csv
drwxrwxr-x 4 ubuntu ubuntu 4.0K Aug  2 00:00 train2/
drwxrwxr-x 2 ubuntu ubuntu 4.0K Aug  2 00:00 test2/
-rw-rw-r-- 1 ubuntu ubuntu  14K Aug  2 09:52 graduation_project.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 9.7G Aug  2 10:03 gap_InceptionResNetV2.h5


In [32]:
rm -rf /home/ubuntu/.kaggle/competitions/dogs-vs-cats-redux-kernels-edition/*.zip

In [29]:
ls -ltrh /home/ubuntu/.kaggle/competitions/dogs-vs-cats-redux-kernels-edition/*.zip

-rw-rw-r-- 1 ubuntu ubuntu 272M Aug  1 23:47 /home/ubuntu/.kaggle/competitions/dogs-vs-cats-redux-kernels-edition/test.zip
-rw-rw-r-- 1 ubuntu ubuntu 544M Aug  1 23:47 /home/ubuntu/.kaggle/competitions/dogs-vs-cats-redux-kernels-edition/train.zip


In [31]:
ls -ltrh /home/ubuntu/

total 1.1G
-rw-rw-r--  1 ubuntu ubuntu 294M May 25  2017 gap_Xception.h5
-rw-rw-r--  1 ubuntu ubuntu  74M May 25  2017 gap_VGG19.h5
-rw-rw-r--  1 ubuntu ubuntu  74M May 25  2017 gap_VGG16.h5
-rw-rw-r--  1 ubuntu ubuntu 294M May 25  2017 gap_ResNet50.h5
-rw-rw-r--  1 ubuntu ubuntu 294M May 25  2017 gap_InceptionV3.h5
-rw-rw-r--  1 ubuntu ubuntu 2.8M Jul 18 02:00 Nvidia_Cloud_EULA.pdf
drwxrwxr-x  5 ubuntu ubuntu 4.0K Jul 19 18:10 tutorials/
drwxrwxr-x  8 ubuntu ubuntu 4.0K Jul 19 18:10 examples/
-rw-rw-r--  1 ubuntu ubuntu 2.8K Jul 19 18:11 README
drwxrwxr-x 24 ubuntu ubuntu 4.0K Jul 19 19:15 anaconda3/
drwxrwxr-x 10 ubuntu ubuntu 4.0K Jul 19 19:59 src/
-rw-rw-r--  1 ubuntu ubuntu 112K Aug  1 23:46 sample_submission.csv
drwxrwxr-x  7 ubuntu ubuntu 4.0K Aug  2 10:07 uda/


In [ ]:

def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    x = Input((height, width, 3))
    if lambda_func:
        x = Lambda(lambda_func)(x)

    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("train2", image_size, shuffle=False, 
                                              batch_size=16)
    test_generator = gen.flow_from_directory("test2", image_size, shuffle=False, 
                                             batch_size=16, class_mode=None)
    print("stage2")

    #my add starts
    train_filenames = train_generator.filenames
    train_nb_samples = len(train_filenames)

    test_filenames = test_generator.filenames
    test_nb_samples = len(test_filenames)
    
    #my add finish
    
    print("{}".format(MODEL.__name__))
    print("stage3")
    
    train = model.predict_generator(train_generator,steps=train_nb_samples,verbose=1)
    test = model.predict_generator(test_generator,steps=test_nb_samples,verbose=1)
    
    print("stage4")

                                   
    with h5py.File("gap_%s.h5"%MODEL.__name__) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)
        
write_gap(ResNet50, (224, 224), resnet50.preprocess_input)
        
'''
write_gap(InceptionResNetV2, (299, 299), inception_resnet_v2.preprocess_input)


write_gap(ResNet50, (224, 224), resnet50.preprocess_input)
write_gap(InceptionV3, (299, 299), inception_v3.preprocess_input)
write_gap(Xception, (299, 299), xception.preprocess_input)
'''


Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
stage2
ResNet50
stage3
  290/25000 [..............................] - ETA: 1:00:45

In [6]:
print("test")

test


In [3]:
K.tensorflow_backend._get_available_gpus()

[]

In [11]:
import tensorflow as tf
import keras
config = tf.ConfigProto( device_count = {'GPU':1,'CPU':4})
sess = tf.Session(config=config)
keras.backend.set_session(sess)

In [13]:
K.tensorflow_backend._get_available_gpus()


[]

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11268157532926125419
]


In [2]:
import tensorflow as tf

if tf.test.gpu_device_name():
    print("{}".format(tf.test.gpu_device_name()))
else:
    print("plz install gpu version")

/device:GPU:0


NameError: name 'input_model' is not defined

In [5]:
from keras.applications.inception_resnet_v2 import preprocess_input

img_width, img_height = 299,299

epochs = 10
batch_size = 16

base_model = Sequential()
base_model.add(Conv2D(32,(3,3), input_shape=get_input_shape(img_width,img_height)))
base_model.add(Activation('relu'))
base_model.add(MaxPooling2D(pool_size=(2,2)))

base_model.add(Conv2D(32,(3,3)))
base_model.add(Activation('relu'))
base_model.add(MaxPooling2D(pool_size=(2,2)))

base_model.add(Conv2D(64,(3,3)))
base_model.add(Activation('relu'))
base_model.add(MaxPooling2D(pool_size=(2,2)))

base_model.add(Flatten())
base_model.add(Dense(64))
base_model.add(Activation('relu'))
base_model.add(Dropout(0.5))
base_model.add(Dense(1))
base_model.add(Activation('sigmoid'))

base_model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

#seems rescale is very important.
#image_gen = ImageDataGenerator(featurewise_center=False,fill_mode='nearest',
#                               cval=0,horizontal_flip=True,rotation_range=20,shear_range=0.2,rescale=1./255)



image_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

# validation_split=0.4, cval=0, fill_mode='constant'

train_gen = image_gen.flow_from_directory("train", (img_width,img_height), shuffle=True,interpolation='bicubic',
                                    batch_size=16)# subset = "training",class_mode="categorical"

validation_gen = image_gen.flow_from_directory("validation", (img_width,img_height), shuffle=False,interpolation='bicubic',
                                    batch_size=16) # 'binary',

test_gen = image_gen.flow_from_directory("test2", (img_width,img_height), shuffle=False, 
                                         batch_size=8, class_mode=None)


Found 16995 images belonging to 2 classes.
Found 7999 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
